## lightGBM(Reg) + tf-idf(max) + sudachi(C)

In [1]:
# setup lib
! pip install scikit-learn optuna lightgbm pandas
# optuna/lightgbm(gpu)
#! pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 44.7 MB/s 
     |████████████████████████████████| 81 kB 11.1 MB/s 
     |████████████████████████████████| 210 kB 116.5 MB/s 
     |████████████████████████████████| 78 kB 9.1 MB/s 
     |████████████████████████████████| 147 kB 64.1 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
     |████████████████████████████████| 112 kB 80.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=e350e2d8a21254e79f0e8289264ac06e1c3c108f7226f7b50472c681d2eb97fc
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0


In [40]:
# ライブラリの導入

import numbers
import numpy as np
import pandas as pd

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
import scipy as sp
from functools import partial

def adapt_labels(labels):
  max_label = np.max(labels) # 2 or 4

  if max_label == 2:
    labels += 2
  elif max_label == 4:
    labels -= 2
  else:
    print('Exception error! please check with np.unique(labels).')
  return labels

In [3]:
#ファイル
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/grad_comp

Mounted at /content/drive
/content/drive/MyDrive/grad_comp


## load data

In [4]:
# preprocess path
prepro = 'sudachi/C'
# n_features = 10000

In [5]:
# data

f = open('preprocess/' + prepro + '/text.prep_train.txt', 'r')
train_data = f.read()
train_data = train_data.split('\n')
del train_data[30000]
#x_train = np.zeros((30000, len(res)))

f = open('preprocess/' + prepro + '/text.prep_test.txt', 'r')
test_data = f.read()
test_data = test_data.split('\n')
del test_data[2500]
#x_test = np.zeros((2500, len(res)))

f = open('preprocess/' + prepro + '/text.prep_dev.txt', 'r')
dev_data = f.read()
dev_data = dev_data.split('\n')
del dev_data[2500]
#x_dev = np.zeros((2500, len(res)))

# label
y_train = np.loadtxt('/content/drive/MyDrive/grad_comp/data/label.train.txt')
y_dev = np.loadtxt('/content/drive/MyDrive/grad_comp/data/label.dev.txt')
adapt_labels(y_train)
adapt_labels(y_dev)

u, counts = np.unique(y_train, return_counts=True)
print(u)
print(counts)

[0. 1. 2. 3. 4.]
[3543 5593 9227 7760 3877]


## TF-IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(train_data)
x_dev = vectorizer.transform(dev_data)
x_test = vectorizer.transform(test_data)

In [7]:
x_train.shape

(30000, 26803)

QWK = 0.041  C = 0.01
QWK = 0.210  C = 0.1
QWK = 0.334  C = 1
QWK = 0.324  C = 10
QWK = 0.290  C = 100
最適なハイパーパラメタは C = 1

## lightGBM

In [83]:
!pip install scipy==1.3.0
import scipy as sp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 23.6 MB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scipy: filename=scipy-1.3.0-cp38-cp38-linux_x86_64.whl size=43877168 sha256=04a23cebc77bce31e2d4a3957de61e5dd3ec00712db855d2f38ffcd761759d92
  Stored in directory: /root/.cache/pip/wheels/2c/c1/a0/d7b4dc8a7b3c8e3cfa99b2098af0c94aadce5f5b516015d952
Successfully built scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.3.0 which is incompatible.
pymc 4.1.4

You must restart the runtime in order to use newly installed versions.

In [84]:
sp.version.full_version

'1.7.3'

In [80]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X) # Xにも影響しないよう
        for i, pred in enumerate(X_p):
          # こちらはエラーなし
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        # ライブラリののqwkに変更
        ll = cohen_kappa_score(y,X_p,weights='quadratic')
        return -ll # optimize.minimizeより

    # 閾値の最適化(nelder-mead法)
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        x_qwk = sp.optimize.minimize(loss_partial, initial_coef,method='Nelder-Mead')
        #print(x_qwk)
        
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef,method='Nelder-Mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [81]:
def lgb_custom_metric_qwk_regression(preds, data):
    """LightGBM のカスタムメトリックを計算する関数

    回帰問題として解いた予測から QWK を計算する"""
    # 正解ラベル
    y_true = data.get_label()
    # 1:予測ラベル
    #y_pred = np.clip(preds, 0, 7).round()  # 単純に予測値を clip して四捨五入する
    # 2:予測ラベル(最適化)
    optR = OptimizedRounder()
    optR.fit(preds, y_true)
    optimized = optR.predict(preds, optR.coefficients)
    # QWK を計算する
    return 'qwk', qwk(y_true, optimized), True

# # 閾値の最適化
# optR = OptimizedRounder()
# optR.fit(regression_predictions, original_labels)
# print(optR.coefficients)

# optimized = optR.predict(regression_predictions, optR.coefficients)
# # 最適化した値でqwk出力
# optimized_qwk_dev = cohen_kappa_score(y_dev,optimized,weights='quadratic')
# print(optimized_qwk_dev)

def qwk(y_true, y_pred):
    """QWK (Quadratic Weighted Kappa) を計算する関数"""
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

In [82]:
# 検証用

# Dataset形式に変換
# train_set = lgb.Dataset(X_train, y_train)
# dev_set = lgb.Dataset(X_dev, y_dev)

params = {
    "objective" : "multiclass",
    "metric" : "multi_logloss",
    "num_class" : 5
}

# # データセットを読み込む
# train_df = pd.read_csv('train.csv.zip')

# # 説明変数
# x_train = train_df.drop('Response', axis=1)

# # 目的変数
# y_train = train_df.Response.astype(float)
# # 目的変数はゼロからスタートにする
# y_train -= 1

# LightGBM のデータセット表現にする
lgb_train = lgb.Dataset(x_train, y_train)

# 回帰問題として解く
lgbm_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'first_metric_only': True,
    'verbose': -1, # 学習中のコマンドライン出力
}

# データ分割の方法
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 5-Fold CV で検証する
result = lgb.cv(lgbm_params,
                lgb_train,
                num_boost_round=1000,
                early_stopping_rounds=100,
                folds=folds,
                verbose_eval=10,
                feval=lgb_custom_metric_qwk_regression,
                seed=42,
                )

# early stop したラウンドでの QWK を出力する
print(f'CV Mean QWK: {result["qwk-mean"][-1]}')

 final_simplex: (array([[0.5       , 1.5       , 2.5       , 3.5       ],
       [0.50001221, 1.5       , 2.5       , 3.5       ],
       [0.5       , 1.50003662, 2.5       , 3.5       ],
       [0.5       , 1.5       , 2.50006104, 3.5       ],
       [0.5       , 1.5       , 2.5       , 3.50008545]]), array([-0., -0., -0., -0., -0.]))
           fun: -0.0
       message: 'Optimization terminated successfully.'
          nfev: 71
           nit: 12
        status: 0
       success: True
             x: array([0.5, 1.5, 2.5, 3.5])
{}


TypeError: ignored

In [27]:
# Dataset形式に変換
# train_set = lgb.Dataset(X_train, y_train)
# dev_set = lgb.Dataset(X_dev, y_dev)

params = {
    "device": "gpu",
    "objective" : "multiclass",
    "metric" : "multi_logloss",
    "num_class" : 5
}

# # データセットを読み込む
# train_df = pd.read_csv('train.csv.zip')

# # 説明変数
# x_train = train_df.drop('Response', axis=1)

# # 目的変数
# y_train = train_df.Response.astype(float)
# # 目的変数はゼロからスタートにする
# y_train -= 1

# LightGBM のデータセット表現にする
lgb_train = lgb.Dataset(x_train, y_train)

# 回帰問題として解く
lgbm_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'first_metric_only': True,
    'verbose': -1, # 学習中のコマンドライン出力
}

# データ分割の方法
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 5-Fold CV で検証する
result = lgb.cv(lgbm_params,
                lgb_train,
                num_boost_round=1000,
                early_stopping_rounds=100,
                folds=folds,
                verbose_eval=10,
                feval=lgb_custom_metric_qwk_regression,
                seed=42,
                )

# early stop したラウンドでの QWK を出力する
print(f'CV Mean QWK: {result["qwk-mean"][-1]}')

 final_simplex: (array([[0.5       , 1.5       , 2.5       , 3.5       ],
       [0.50001221, 1.5       , 2.5       , 3.5       ],
       [0.5       , 1.50003662, 2.5       , 3.5       ],
       [0.5       , 1.5       , 2.50006104, 3.5       ],
       [0.5       , 1.5       , 2.5       , 3.50008545]]), array([-0., -0., -0., -0., -0.]))
           fun: -0.0
       message: 'Optimization terminated successfully.'
          nfev: 71
           nit: 12
        status: 0
       success: True
             x: array([0.5, 1.5, 2.5, 3.5])


TypeError: ignored

non optimizer
[10]	cv_agg's rmse: 1.1306 + 0.00219238	cv_agg's qwk: 0.093538 + 0.00473876
[20]	cv_agg's rmse: 1.10722 + 0.00316532	cv_agg's qwk: 0.14753 + 0.00792863
[30]	cv_agg's rmse: 1.09398 + 0.00297894	cv_agg's qwk: 0.189634 + 0.00754283
[40]	cv_agg's rmse: 1.08554 + 0.00290599	cv_agg's qwk: 0.22336 + 0.00342504
[50]	cv_agg's rmse: 1.07955 + 0.00305654	cv_agg's qwk: 0.240559 + 0.00416166
[60]	cv_agg's rmse: 1.07542 + 0.00269958	cv_agg's qwk: 0.255057 + 0.0050241
[70]	cv_agg's rmse: 1.07217 + 0.00280997	cv_agg's qwk: 0.265979 + 0.00527485
[80]	cv_agg's rmse: 1.06975 + 0.00291788	cv_agg's qwk: 0.274272 + 0.00731139
[90]	cv_agg's rmse: 1.06779 + 0.00271506	cv_agg's qwk: 0.282002 + 0.00850304
[100]	cv_agg's rmse: 1.06637 + 0.00274278	cv_agg's qwk: 0.288859 + 0.00906319
[110]	cv_agg's rmse: 1.0653 + 0.00272203	cv_agg's qwk: 0.293129 + 0.00809321
[120]	cv_agg's rmse: 1.06448 + 0.00291496	cv_agg's qwk: 0.297944 + 0.00914024
[130]	cv_agg's rmse: 1.06359 + 0.00308441	cv_agg's qwk: 0.302275 + 0.00884944
[140]	cv_agg's rmse: 1.0633 + 0.00296652	cv_agg's qwk: 0.305308 + 0.00927883
[150]	cv_agg's rmse: 1.06284 + 0.00300857	cv_agg's qwk: 0.308474 + 0.00910171
[160]	cv_agg's rmse: 1.0622 + 0.00300849	cv_agg's qwk: 0.3123 + 0.0090972
[170]	cv_agg's rmse: 1.06207 + 0.00317831	cv_agg's qwk: 0.316603 + 0.00880866
[180]	cv_agg's rmse: 1.06201 + 0.00327053	cv_agg's qwk: 0.320307 + 0.0107394
[190]	cv_agg's rmse: 1.06203 + 0.00345727	cv_agg's qwk: 0.322476 + 0.0110931
[200]	cv_agg's rmse: 1.06208 + 0.00346878	cv_agg's qwk: 0.324237 + 0.0111002
[210]	cv_agg's rmse: 1.06205 + 0.00352508	cv_agg's qwk: 0.325391 + 0.00950517
[220]	cv_agg's rmse: 1.06203 + 0.00353636	cv_agg's qwk: 0.326758 + 0.00771311
[230]	cv_agg's rmse: 1.06227 + 0.00354542	cv_agg's qwk: 0.328505 + 0.00745911
[240]	cv_agg's rmse: 1.06224 + 0.00360543	cv_agg's qwk: 0.330838 + 0.00779444
[250]	cv_agg's rmse: 1.06242 + 0.00380436	cv_agg's qwk: 0.332008 + 0.00753289
[260]	cv_agg's rmse: 1.06266 + 0.00373026	cv_agg's qwk: 0.332632 + 0.00730913
[270]	cv_agg's rmse: 1.06282 + 0.00372533	cv_agg's qwk: 0.334488 + 0.00657479
CV Mean QWK: 0.31870013389412544

In [ ]:
# 閾値の最適化
optR = OptimizedRounder()
optR.fit(regression_predictions, original_labels)
print(optR.coefficients)

optimized = optR.predict(regression_predictions, optR.coefficients)
# 最適化した値でqwk出力
optimized_qwk_dev = cohen_kappa_score(y_dev,optimized,weights='quadratic')
print(optimized_qwk_dev)

In [8]:
# best_qwk = 0
# best_c = 1
# for c in [1, 5,10, 50]:
#     model = LogisticRegression(C=c, random_state=0,max_iter=1500)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_dev)
#     qwk = cohen_kappa_score(y_dev,y_pred, weights='quadratic')
#     if qwk > best_qwk:
#         best_qwk = qwk
#         best_c = c
#     print("QWK = %.3f  C = %s" % (qwk, str(c)))
# print("最適なハイパーパラメタは C = %s" % str(best_c))


# model = LogisticRegression(C=best_c, random_state=0,max_iter=1500)
# model.fit(X_train, y_train)
# y_preds = model.predict(X_test)

# y_preds -= 2
# with open('src/' + prepro + '/eval.txt','w') as f:
#   for y_pred in y_preds:
#     y_pred = int(y_pred)
#     y_pred = str(y_pred)
#     f.write(y_pred + '\n')

QWK = 0.330  C = 1
QWK = 0.338  C = 5
QWK = 0.324  C = 10
QWK = 0.299  C = 50
最適なハイパーパラメタは C = 5


-2 origin(nagisa)
QWK = 0.329  C = 1
QWK = 0.339  C = 5
QWK = 0.333  C = 10
QWK = 0.294  C = 50
最適なハイパーパラメタは C = 5

0 origin(顔文字あり)


In [14]:
print(f'最適なハイパラ C = {str(best_c)} : {prepro}')

最適なハイパラ C = 5 : sudachi/C


## submit

In [ ]:
y_preds = model.predict(X_test)

with open('src/' + prepro + '/eval.txt','w') as f:
  for y_pred in y_preds:
    y_pred = int(y_pred)
    y_pred = str(y_pred)
    f.write(y_pred + '\n')